In [1]:
import pandas as pd
import numpy as np
import pickle
import re

GAdf = pd.read_csv('GA_LISTINGS_SALES_V2.csv')

In [2]:
GAdf = GAdf[GAdf.details.str.contains("etached")]
GAdf['baths_half'].fillna(0, inplace = True)
GAdf['baths_full'].fillna(0, inplace=True)
GAdf['beds'].dropna(inplace=True)

In [3]:
GAdf.rename(columns = {'zip':'zipcode'}, inplace=True)
GAdf.drop('Unnamed: 0', inplace=True, axis=1)

In [4]:
## acres was given as square footage instead.
GAdf.loc[31032, 'details'] = 'Detached, 3 Beds, 2½ Baths, 1 Acres'
## Take away CA zipcode
GAdf=GAdf[GAdf.zipcode!='92544'].copy()

In [5]:
pd.set_option('display.max_columns', 50)

In [6]:
# Regex only updates some values that were before rounded (e.g. acres). It does not add new values
# For this reason, it will probably not be used

# acres = GAdf[GAdf.details.str.contains('Acres')]
# squarefootage = GAdf[GAdf.details.str.contains('Sq Ft')]
# acsplit = acres.details.str.split('((\d)*(\.?)*(\d)+) Acre')
# sqftsplit = squarefootage.details.str.split('((\d)*(\.?)*(\d)+) Sq Ft')
# for row in acres.index:
#     GAdf.loc[row, 'lot_size'] = float(acsplit[row][-5])
    
# for row in squarefootage.index:
#     GAdf.loc[row, 'square_footage'] = float(sqftsplit[row][-5])

In [7]:
GAdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12244 entries, 1 to 31032
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   latitude                  12244 non-null  float64
 1   longitude                 12244 non-null  float64
 2   full_street_address       12244 non-null  object 
 3   city                      12244 non-null  object 
 4   county_name               12244 non-null  object 
 5   beds                      12097 non-null  float64
 6   baths_full                12244 non-null  float64
 7   baths_half                12244 non-null  float64
 8   square_footage            10512 non-null  float64
 9   lot_size                  11138 non-null  float64
 10  year_built                12202 non-null  float64
 11  details                   12244 non-null  object 
 12  special_features          12244 non-null  int64  
 13  unit_count                149 non-null    float64
 14  price 

In [8]:
abz = GAdf.groupby('zipcode').agg({'square_footage':'mean', 'lot_size':'mean'}).reset_index()

In [9]:
## these zips likely have too few observations

abz.loc[abz['square_footage'].isnull()]

,zipcode,square_footage,lot_size
301,30634,NaN,9.0
342,30755,NaN,0.0
345,30810,NaN,76.0
354,30906,NaN,0.0
375,31028,NaN,0.0
389,31058,NaN,209.0
395,31065,NaN,1.0
396,31066,NaN,3.0
399,31070,NaN,0.0
400,31072,NaN,5.0


In [10]:
def fix_sqft(df, zipcode, abz):
    if zipcode in list(df.zipcode.values) and \
     abz.loc[abz.zipcode == zipcode, 'square_footage'].isnull().values[0] == False:
        return abz.loc[abz.zipcode==zipcode, 'square_footage'].values[0]
    else: pass

    
def fix_lotsize(df, zipcode, abz):    
    if zipcode in list(abz.zipcode.values) and \
    abz.loc[abz.zipcode == zipcode, 'lot_size'].isnull().values[0] == False:
        return abz.loc[abz.zipcode==zipcode, 'lot_size'].values[0]
    else: pass

In [11]:
sqftnull = GAdf.loc[GAdf['square_footage'].isnull()].copy()
lotnull = GAdf.loc[GAdf['lot_size'].isnull()].copy()

In [12]:
# abz.loc[abz.zipcode == '31058', 'square_footage'].isnull().values
# GAdf.loc[GAdf['zipcode']=='31058']

In [13]:
sqftnull['square_footage']= sqftnull.apply(lambda row: fix_sqft(GAdf, row['zipcode'], abz), axis=1)
lotnull['lot_size'] = lotnull.apply(lambda row: fix_lotsize(GAdf, row['zipcode'], abz), axis =1)

In [14]:
for row in sqftnull.index:
    GAdf.loc[row, 'square_footage'] = sqftnull.loc[row, 'square_footage']

for row in lotnull.index:
    GAdf.loc[row, 'lot_size'] = lotnull.loc[row, 'lot_size']

In [15]:
GAdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12244 entries, 1 to 31032
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   latitude                  12244 non-null  float64
 1   longitude                 12244 non-null  float64
 2   full_street_address       12244 non-null  object 
 3   city                      12244 non-null  object 
 4   county_name               12244 non-null  object 
 5   beds                      12097 non-null  float64
 6   baths_full                12244 non-null  float64
 7   baths_half                12244 non-null  float64
 8   square_footage            12221 non-null  float64
 9   lot_size                  11956 non-null  float64
 10  year_built                12202 non-null  float64
 11  details                   12244 non-null  object 
 12  special_features          12244 non-null  int64  
 13  unit_count                149 non-null    float64
 14  price 

In [16]:
hsdf = pd.read_csv('high_schools.csv')
msdf = pd.read_csv('middle_schools.csv')
esdf = pd.read_csv('elementary_schools.csv')

In [17]:
## renaming lat/long in case we ever use it. For now, however, we won't.

hsdf.rename(columns = {'rating': 'HS_rating', 'lat':'lat_h', 'long':'long_h'}, inplace=True)
msdf.rename(columns = {'rating': 'MS_rating', 'lat':'lat_m', 'long':'long_m'}, inplace=True)
esdf.rename(columns = {'rating': 'ES_rating', 'lat':'lat_e', 'long':'long_e'}, inplace=True)

In [18]:
hsdf = hsdf.groupby(['zipcode']).agg({'HS_rating':'mean'}).reset_index()
msdf = msdf.groupby(['zipcode']).agg({'MS_rating':'mean'}).reset_index()
esdf = esdf.groupby(['zipcode']).agg({'ES_rating':'mean'}).reset_index()



In [19]:
crime = pd.read_csv('crime_rating_zipcode.csv')
crime.rename(columns={"census_zcta5_geoid": 'zipcode'}, inplace=True)
crime = crime[['overall_crime_grade', 'property_crime_grade','zipcode']]

In [20]:
GAdf['zipcode'] = pd.to_numeric(GAdf['zipcode'])

In [21]:
df = GAdf.merge(crime, on = 'zipcode', how= 'inner')
df = df.merge(hsdf, on = 'zipcode', how= 'inner')
df = df.merge(msdf, on = 'zipcode', how= 'inner')
df = df.merge(esdf, on = 'zipcode', how= 'inner')

In [22]:
df.head(5)

,latitude,longitude,full_street_address,city,county_name,beds,baths_full,baths_half,square_footage,lot_size,year_built,details,special_features,unit_count,price,transaction_type,listing_status,listing_special_features,census_state_name,census_county_name,zipcode,overall_crime_grade,property_crime_grade,HS_rating,MS_rating,ES_rating
0,30.781796,-83.558475,505 S Lee Street,Quitman,Brooks-County,3.0,1.0,0.0,1460.0,0.0,1910.0,"Detached, 3 Beds, 1 Bath, 1,460 Sq Ft",0,NaN,99000,1,1,0,Georgia,"Brooks, GA",31643,D-,D-,2.5,2.0,1.000000
1,30.781796,-83.558475,505 S Lee Street,Quitman,Brooks-County,3.0,1.0,0.0,1460.0,0.0,1910.0,"Detached, 3 Beds, 1 Bath, 1,460 Sq Ft",0,NaN,99000,1,1,0,Georgia,"Brooks, GA",31643,D-,D-,2.5,2.0,1.000000
2,30.762972,-81.660240,84 Whippoorwill Circle,Kingsland,Camden-County,3.0,2.0,0.0,1618.0,0.0,1986.0,"Detached, 3 Beds, 2 Baths, 1,618 Sq Ft",0,NaN,200000,1,1,0,Georgia,"Camden, GA",31548,B-,C+,8.0,6.0,7.333333
3,30.804209,-81.653325,101 College Street,Kingsland,Camden-County,4.0,2.0,0.0,2103.0,0.0,2020.0,"Detached, 4 Beds, 2 Baths, 2,103 Sq Ft",2,NaN,339900,1,1,2,Georgia,"Camden, GA",31548,B-,C+,8.0,6.0,7.333333
4,30.823195,-81.635187,241 Jake Colton Drive,Kingsland,Camden-County,4.0,3.0,0.0,2954.0,0.0,2019.0,"Detached, 4 Beds, 3 Baths, 2,954 Sq Ft",0,NaN,679900,1,1,0,Georgia,"Camden, GA",31548,B-,C+,8.0,6.0,7.333333


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10678 entries, 0 to 10677
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   latitude                  10678 non-null  float64
 1   longitude                 10678 non-null  float64
 2   full_street_address       10678 non-null  object 
 3   city                      10678 non-null  object 
 4   county_name               10678 non-null  object 
 5   beds                      10548 non-null  float64
 6   baths_full                10678 non-null  float64
 7   baths_half                10678 non-null  float64
 8   square_footage            10669 non-null  float64
 9   lot_size                  10482 non-null  float64
 10  year_built                10644 non-null  float64
 11  details                   10678 non-null  object 
 12  special_features          10678 non-null  int64  
 13  unit_count                132 non-null    float64
 14  price 

In [24]:
fmr = pd.read_csv('fmrClean.csv')

In [25]:
df = df.drop('unit_count', axis=1)

In [26]:
df['HS_rating'].fillna(np.mean(df.HS_rating), inplace=True)
df['MS_rating'].fillna(np.mean(df.MS_rating), inplace=True)
df['ES_rating'].fillna(np.mean(df.ES_rating), inplace=True)

In [27]:
np.mean(df.HS_rating)

5.3435617130282935

In [28]:
df.dropna(subset = ['beds', 'lot_size', 'square_footage'], inplace=True)

In [29]:
df['rent'] = 0

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10343 entries, 0 to 10676
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   latitude                  10343 non-null  float64
 1   longitude                 10343 non-null  float64
 2   full_street_address       10343 non-null  object 
 3   city                      10343 non-null  object 
 4   county_name               10343 non-null  object 
 5   beds                      10343 non-null  float64
 6   baths_full                10343 non-null  float64
 7   baths_half                10343 non-null  float64
 8   square_footage            10343 non-null  float64
 9   lot_size                  10343 non-null  float64
 10  year_built                10314 non-null  float64
 11  details                   10343 non-null  object 
 12  special_features          10343 non-null  int64  
 13  price                     10343 non-null  int64  
 14  transa

In [31]:
def rents(beds, fmr, county):
    if beds <=4:
        rate = fmr.loc[fmr.county_name==county, 'fmr_'+str(int(beds))].values[0]
    else: rate = fmr.loc[fmr.county_name==county, 'fmr_4'].values[0]*(1.15**(beds-4))
    return rate

In [32]:
df['rent']=df.apply(lambda row: rents(row['beds'], fmr, row['county_name']), axis=1)

In [33]:
## estimate prior to deductions

df['caprate'] = 100 * df['rent']/df['price']

In [34]:
df.head(5)

,latitude,longitude,full_street_address,city,county_name,beds,baths_full,baths_half,square_footage,lot_size,year_built,details,special_features,price,transaction_type,listing_status,listing_special_features,census_state_name,census_county_name,zipcode,overall_crime_grade,property_crime_grade,HS_rating,MS_rating,ES_rating,rent,caprate
0,30.781796,-83.558475,505 S Lee Street,Quitman,Brooks-County,3.0,1.0,0.0,1460.0,0.0,1910.0,"Detached, 3 Beds, 1 Bath, 1,460 Sq Ft",0,99000,1,1,0,Georgia,"Brooks, GA",31643,D-,D-,2.5,2.0,1.000000,1219.0,1.231313
1,30.781796,-83.558475,505 S Lee Street,Quitman,Brooks-County,3.0,1.0,0.0,1460.0,0.0,1910.0,"Detached, 3 Beds, 1 Bath, 1,460 Sq Ft",0,99000,1,1,0,Georgia,"Brooks, GA",31643,D-,D-,2.5,2.0,1.000000,1219.0,1.231313
2,30.762972,-81.660240,84 Whippoorwill Circle,Kingsland,Camden-County,3.0,2.0,0.0,1618.0,0.0,1986.0,"Detached, 3 Beds, 2 Baths, 1,618 Sq Ft",0,200000,1,1,0,Georgia,"Camden, GA",31548,B-,C+,8.0,6.0,7.333333,1380.0,0.690000
3,30.804209,-81.653325,101 College Street,Kingsland,Camden-County,4.0,2.0,0.0,2103.0,0.0,2020.0,"Detached, 4 Beds, 2 Baths, 2,103 Sq Ft",2,339900,1,1,2,Georgia,"Camden, GA",31548,B-,C+,8.0,6.0,7.333333,1603.0,0.471609
4,30.823195,-81.635187,241 Jake Colton Drive,Kingsland,Camden-County,4.0,3.0,0.0,2954.0,0.0,2019.0,"Detached, 4 Beds, 3 Baths, 2,954 Sq Ft",0,679900,1,1,0,Georgia,"Camden, GA",31548,B-,C+,8.0,6.0,7.333333,1603.0,0.235770


In [35]:
df.sort_values(by='caprate', ascending = False)
## these will either need to be taken care of by hand or other features needed

,latitude,longitude,full_street_address,city,county_name,beds,baths_full,baths_half,square_footage,lot_size,year_built,details,special_features,price,transaction_type,listing_status,listing_special_features,census_state_name,census_county_name,zipcode,overall_crime_grade,property_crime_grade,HS_rating,MS_rating,ES_rating,rent,caprate
4069,32.657072,-84.006595,1175 Zenith Mill Road,Roberta,Crawford-County,3.0,2.0,0.0,1320.000000,1.0,1938.0,"Detached, 3 Beds, 2 Baths, 1,320 Sq Ft",0,1,1,1,0,Georgia,"Crawford, GA",31078,C-,D+,4.0,5.0,3.000000,1.207000e+03,120700.000000
2446,33.989071,-83.465485,100 Fair Oaks Drive,Athens,Clarke-County,71.0,56.0,0.0,30288.000000,0.0,2001.0,"Detached, 71 Beds, 56 Baths, 30,288 Sq Ft",0,4500000,1,1,0,Georgia,"Clarke, GA",30606,D,D,3.0,3.5,4.600000,1.769053e+07,393.122925
8717,33.452389,-84.093109,171 Country Meadows Court,McDonough,Henry-County,3.0,2.0,0.0,1944.000000,0.0,1995.0,"Detached, 3 Beds, 2 Baths, 1,944 Sq Ft",0,2145,1,0,0,Georgia,"Henry, GA",30252,A-,B,7.0,6.5,6.200000,1.890000e+03,88.111888
9180,34.111433,-83.614939,2214 Savannah Drive,Jefferson,Jackson-County,4.0,2.0,1.0,2730.104478,0.0,2021.0,"Detached, 4 Beds, 2½ Baths",2,1995,1,1,2,Georgia,"Jackson, GA",30549,C+,D+,7.5,7.0,7.000000,1.351000e+03,67.719298
695,32.852852,-83.690353,3796 Grand Avenue,MacOn,Bibb-County,3.0,1.0,0.0,1142.000000,0.0,1924.0,"Detached, 3 Beds, 1 Bath, 1,142 Sq Ft",0,8000,1,1,0,Georgia,"Bibb, GA",31204,D-,D-,3.0,3.0,3.500000,1.207000e+03,15.087500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5929,34.248541,-84.054929,0 Browns Bridge Road #9004867,Cumming,Forsyth-County,3.0,2.0,1.0,3689.275362,54.0,1972.0,"Detached, 3 Beds, 2½ Baths, 54.00 Acres",0,13500000,1,1,0,Georgia,"Forsyth, GA",30041,A-,B-,9.0,7.5,7.857143,1.890000e+03,0.014000
5928,34.249393,-84.052846,5325 Browns Bridge Road,Cumming,Forsyth-County,3.0,2.0,1.0,2241.000000,54.0,1972.0,"Detached, 3 Beds, 2½ Baths, 2,241 Sq Ft",0,13500000,1,1,0,Georgia,"Forsyth, GA",30041,A-,B-,9.0,7.5,7.857143,1.890000e+03,0.014000
10407,34.843967,-83.719654,7320 Chestnut Valley Road,Hiawassee,Towns-County,6.0,6.0,0.0,1950.782609,457.0,1889.0,"Detached, 6 Beds, 6 Baths, 457.00 Acres",0,15000000,1,1,0,Georgia,"Towns, GA",30546,D+,C,7.0,7.0,7.000000,1.803890e+03,0.012026
10374,34.843967,-83.719654,7320 Chestnut Valley Road,Hiawassee,Towns-County,6.0,6.0,0.0,1950.782609,457.0,1889.0,"Detached, 6 Beds, 6 Baths, 457.00 Acres",0,15000000,1,1,0,Georgia,"Towns, GA",30546,D+,C,7.0,7.0,7.000000,1.803890e+03,0.012026


In [36]:
df = df.drop(index = 2446) ## Certainly not single-family

In [37]:
df = df.loc[~df.year_built.isnull()]

In [39]:
df.shape

(10313, 27)

In [40]:
df.to_csv('cleaned.csv')